In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
import matplotlib.pyplot as plt
%matplotlib inline

#### The data used

In [3]:
# #df = pd.read_csv('data/anemia_synth_dataset_hb_some_nans.csv')
# df = pd.read_csv('data/anemia_synth_dataset_with_unspecified.csv')
# df = df.fillna(0)
# # classes = list(df.label.unique())
# # nums = [i for i in range(len(classes))]
# # class_dict = dict(zip(classes, nums))
# class_dict = {'No anemia': 0, 'Hemolytic anemia': 1, 'Aplastic anemia': 2, 'Iron deficiency anemia': 3,
#               'Vitamin B12/Folate deficiency anemia': 4, 'Anemia of chronic disease': 5, 'Unspecified anemia': 6}
# df['label'] = df['label'].replace(class_dict)
# X = df.iloc[:, 0:-1]
# y = df.iloc[:, -1]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
# X_train, y_train = np.array(X_train), np.array(y_train)
# X_test, y_test = np.array(X_test), np.array(y_test)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [6]:
X_test_df = pd.read_csv('../../final/data/test_set_constant.csv')
X_test = X_test_df.iloc[:, 0:-1]
y_test = X_test_df.iloc[:, -1]
X_test, y_test = np.array(X_test), np.array(y_test)
X_test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.783006,112.011007,-1.000000,3.308648,348.014906,86.577465,-1.000000,4.082935,0,1.617594,131.868365,-1.000000,-1.000000,-1.000000,-1.000000,35.349018,-1.000000,7
1,12.609666,104.475099,5.445972,-1.000000,-1.000000,75.082198,-1.000000,5.038345,1,0.334722,97.568987,83.161880,-1.000000,7.147270,-1.000000,37.828999,-1.000000,3
2,7.940172,4.925400,3.255130,5.766738,209.031102,75.254082,-1.000000,3.165345,1,1.649478,2.082970,-1.000000,-1.000000,-1.000000,-1.000000,23.820517,-1.000000,4
3,7.785840,103.757096,2.874384,3.839597,-1.000000,81.873581,39.638010,2.852876,1,0.637843,127.412065,38.534760,51.437387,28.150784,73.296289,23.357519,-1.000000,5
4,12.359715,51.274141,0.178269,1.233844,474.589534,76.469181,49.839768,4.848900,1,0.621034,137.433107,55.314614,47.302738,3.808612,60.076211,37.079144,10.501658,4


In [7]:
X_test.shape, y_test.shape

((14000, 17), (14000,))

In [8]:
test_df =pd.read_csv('../../final/test_dfs/dqn_test_df_basic_5500000.csv')
test_df.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,5.0,0.0,1.0,-1.0,1.0,"['hematocrit', 'gender', 'mcv', 'ret_count', '...",7.0,7.0
1,6.0,1.0,0.0,-1.0,1.0,"['hematocrit', 'gender', 'mcv', 'ferritin', 't...",3.0,7.0
2,5.0,2.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ferritin', 'I...",4.0,4.0
3,5.0,3.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ret_count', '...",5.0,5.0
4,6.0,4.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ferritin', 't...",4.0,4.0


In [9]:
def get_min_of_array(array, n):
    '''The n least elements in an array'''
    idx = np.argpartition(array, n)
    min_vals = array[idx[:n]]
    return min_vals

In [10]:
# X_test_df = pd.concat([pd.DataFrame(X_test), pd.DataFrame(y_test)], axis=1)
# X_test_df.columns = df.columns
# X_test_df.head()

In [11]:
a1 = np.array(test_df['y_actual'], dtype=np.float32)
a2 = np.array(X_test_df['label'], dtype=np.float32)
assert np.array_equal(a1, a2)

#### 0 - No Anemia

In [12]:
no_test_df = test_df[test_df.y_actual== 0]
no_x_test_df = X_test_df[X_test_df.label==0]
combined_no_test_df = pd.concat([no_test_df, no_x_test_df], axis=1)
combined_no_test_df.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
20,2.0,20.0,1.0,1.0,0.0,"['hematocrit', 'No anemia']",0.0,0.0,14.357527,-1.000000,...,1,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.000000,43.072581,-1.00000,0
41,3.0,41.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'No anemia']",0.0,0.0,12.980411,-1.000000,...,0,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.000000,38.941233,-1.00000,0
50,3.0,50.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'No anemia']",0.0,0.0,12.508738,-1.000000,...,0,1.580692,26.953005,-1.000000,-1.00000,-1.000000,-1.000000,37.526213,-1.00000,0
70,2.0,70.0,1.0,1.0,0.0,"['hematocrit', 'No anemia']",0.0,0.0,13.751608,-1.000000,...,1,1.841386,71.188182,-1.000000,-1.00000,-1.000000,-1.000000,41.254824,-1.00000,0
72,2.0,72.0,1.0,1.0,0.0,"['hematocrit', 'No anemia']",0.0,0.0,13.983326,183.693645,...,1,1.623076,47.526064,93.465071,48.54357,28.924289,119.257847,41.949977,110.40971,0


In [13]:
combined_no_test_df.y_pred.value_counts()

0.0    1934
7.0      34
5.0      11
3.0       8
6.0       7
1.0       6
Name: y_pred, dtype: int64

In [15]:
df_0 = combined_no_test_df[combined_no_test_df.y_pred==5]
df_0.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
1110,5.0,1110.0,0.0,-1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ret_count', '...",0.0,5.0,12.067067,133.074973,...,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,36.201201,-1.000000,0
4822,5.0,4822.0,0.0,-1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ret_count', '...",0.0,5.0,12.061731,115.513088,...,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,36.185193,-1.000000,0
5043,5.0,5043.0,0.0,-1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ret_count', '...",0.0,5.0,12.084466,-1.000000,...,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,36.253398,-1.000000,0
5627,5.0,5627.0,0.0,-1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ret_count', '...",0.0,5.0,12.003728,452.696616,...,0,0.445645,39.646397,-1.000000,-1.000000,-1.000000,-1.000000,36.011183,-1.000000,0
6022,5.0,6022.0,0.0,-1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ret_count', '...",0.0,5.0,12.001263,175.160936,...,0,1.531779,57.080536,68.971914,42.617873,18.643214,46.295387,36.003790,70.547835,0


In [16]:
df_0.describe()[['hemoglobin', 'gender', 'mcv', 'ret_count']]

,hemoglobin,gender,mcv,ret_count
count,11.000000,11.000000,11.000000,11.000000
mean,12.224520,0.181818,91.274820,3.935787
std,0.399814,0.404520,5.944908,1.309184
min,12.001263,0.000000,80.137404,2.336610
25%,12.026071,0.000000,87.465447,3.032254
50%,12.067067,0.000000,92.892681,3.817957
75%,12.077971,0.000000,95.374395,4.809621
max,13.035286,1.000000,98.623839,5.992682


#### 1 - Vitamin b12

In [17]:
vit_test_df = test_df[test_df.y_actual == 1]
vit_x_test_df = X_test_df[X_test_df.label== 1]
combined_vit_test_df = pd.concat([vit_test_df, vit_x_test_df], axis=1)
combined_vit_test_df.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
12,5.0,12.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",1.0,1.0,8.363249,-1.000000,...,0,1.066354,24.964254,117.646559,-1.000000,6.393712,-1.000000,25.089747,-1.000000,1
19,5.0,19.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",1.0,1.0,12.647502,277.291915,...,1,0.867387,51.916799,108.418585,73.838946,7.631822,79.222036,37.942505,51.190298,1
39,5.0,39.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",1.0,1.0,8.855406,-1.000000,...,0,1.131964,143.711309,-1.000000,3.137595,-1.000000,73.771704,26.566218,99.359444,1
45,5.0,45.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",1.0,1.0,9.734492,399.066382,...,0,0.549935,25.665206,129.592979,30.180260,19.843242,69.527568,29.203475,81.878709,1
48,5.0,48.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",1.0,1.0,9.966403,127.222346,...,0,1.382820,13.616328,-1.000000,-1.000000,-1.000000,-1.000000,29.899208,-1.000000,1


In [18]:
combined_vit_test_df.y_pred.value_counts()

1.0    1795
7.0       4
2.0       2
Name: y_pred, dtype: int64

In [21]:
df_1 = combined_vit_test_df[combined_vit_test_df.y_pred==7]
df_1.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
4029,6.0,4029.0,0.0,-1.0,1.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",1.0,7.0,12.957460,437.537271,...,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,38.872380,-1.000000,1
5252,6.0,5252.0,0.0,-1.0,1.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",1.0,7.0,6.345608,-1.000000,...,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,19.036825,-1.000000,1
7874,6.0,7874.0,0.0,-1.0,1.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",1.0,7.0,6.334332,286.380455,...,0,0.214912,93.250973,105.229261,48.244279,7.533844,73.487029,19.002996,14.164076,1
9230,6.0,9230.0,0.0,-1.0,1.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",1.0,7.0,11.401385,-1.000000,...,1,0.411344,97.226823,-1.000000,-1.000000,-1.000000,-1.000000,34.204156,-1.000000,1


In [22]:
df_1.describe()[['hemoglobin', 'gender', 'mcv', 'segmented_neutrophils']]

,hemoglobin,gender,mcv,segmented_neutrophils
count,4.000000,4.00,4.000000,4.000000
mean,9.259696,0.75,103.128866,0.142340
std,3.430741,0.50,0.951668,0.059876
min,6.334332,0.00,101.863661,0.101124
25%,6.342789,0.75,102.682136,0.108956
50%,8.873497,1.00,103.322235,0.118686
75%,11.790404,1.00,103.768965,0.152070
max,12.957460,1.00,104.007335,0.230863


In [23]:
# a4= vit_test_df[vit_test_df.y_actual != vit_test_df.y_pred]
# a4.head()

#### 2 - Unspecified Anemia

In [24]:
unspec_test_df = test_df[test_df.y_actual == 2]
unspec_x_test_df = X_test_df[X_test_df.label== 2]
combined_unspec_test_df = pd.concat([unspec_test_df, unspec_x_test_df], axis=1)
combined_unspec_test_df.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
5,5.0,5.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",2.0,2.0,6.650576,96.865214,...,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,19.951727,-1.000000,2
6,5.0,6.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",2.0,2.0,7.448639,392.097724,...,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,22.345916,-1.000000,2
8,5.0,8.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",2.0,2.0,12.111603,-1.000000,...,1,1.083396,134.192397,87.526295,51.076357,13.526479,93.942577,36.334809,82.152699,2
18,5.0,18.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",2.0,2.0,6.387051,219.772768,...,1,1.794993,45.872403,87.511535,-1.000000,19.533982,-1.000000,19.161152,-1.000000,2
25,5.0,25.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",2.0,2.0,10.492080,94.784726,...,0,-1.000000,-1.000000,69.532226,72.191666,29.101395,137.509264,31.476239,15.761581,2


In [25]:
combined_unspec_test_df.y_pred.value_counts()

2.0    1793
Name: y_pred, dtype: int64

In [27]:
df_2 = combined_unspec_test_df[combined_unspec_test_df.y_pred==0]
df_2.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label


In [28]:
df_2.describe()[['hemoglobin', 'gender', 'mcv', 'segmented_neutrophils']]

,hemoglobin,gender,mcv,segmented_neutrophils
count,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN


#### 5 - Hemolytic Anemia

In [29]:
hemolytic_test_df = test_df[test_df.y_actual == 5]
hemolytic_x_test_df = X_test_df[X_test_df.label== 5]
combined_hemolytic_test_df = pd.concat([hemolytic_test_df, hemolytic_x_test_df], axis=1)
combined_hemolytic_test_df.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
3,5.0,3.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ret_count', '...",5.0,5.0,7.785840,103.757096,...,1,0.637843,127.412065,38.534760,51.437387,28.150784,73.296289,23.357519,-1.00000,5
13,5.0,13.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ret_count', '...",5.0,5.0,8.728044,430.737145,...,1,0.852954,20.949438,57.912984,74.215404,1.732596,56.203915,26.184133,32.24998,5
16,5.0,16.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ret_count', '...",5.0,5.0,12.198898,-1.000000,...,1,1.002507,48.747584,-1.000000,-1.000000,-1.000000,-1.000000,36.596693,-1.00000,5
21,5.0,21.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ret_count', '...",5.0,5.0,8.885216,139.285624,...,1,1.499990,15.670225,94.860894,32.177589,12.289254,70.883285,26.655647,38.12750,5
24,5.0,24.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ret_count', '...",5.0,5.0,7.943050,403.401609,...,1,0.582343,66.856717,-1.000000,-1.000000,-1.000000,-1.000000,23.829149,-1.00000,5


In [30]:
combined_hemolytic_test_df.y_pred.value_counts()

5.0    1759
1.0      18
7.0      15
3.0      11
4.0       2
Name: y_pred, dtype: int64

In [31]:
df_5 = combined_hemolytic_test_df[combined_hemolytic_test_df.y_pred==7]
df_5.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
253,5.0,253.0,0.0,-1.0,1.0,"['hematocrit', 'gender', 'mcv', 'ferritin', 'I...",5.0,7.0,7.492641,-1.000000,...,1,0.792964,35.828010,-1.000000,-1.000000,-1.000000,-1.000000,22.477924,-1.0,5
1192,5.0,1192.0,0.0,-1.0,1.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",5.0,7.0,8.514079,172.670388,...,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,25.542237,-1.0,5
2251,5.0,2251.0,0.0,-1.0,1.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",5.0,7.0,12.586130,-1.000000,...,1,1.981789,36.822520,121.285386,56.241042,21.170976,100.633819,37.758391,-1.0,5
2629,5.0,2629.0,0.0,-1.0,1.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",5.0,7.0,10.444841,-1.000000,...,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,31.334523,-1.0,5
3215,7.0,3215.0,0.0,-1.0,1.0,"['hematocrit', 'gender', 'mcv', 'ret_count', '...",5.0,7.0,8.085505,-1.000000,...,0,0.382294,95.469652,-1.000000,-1.000000,-1.000000,-1.000000,24.256515,-1.0,5


In [32]:
df_5.describe()[['hemoglobin', 'gender', 'mcv', 'ret_count']]

,hemoglobin,gender,mcv,ret_count
count,15.000000,15.000000,15.000000,15.000000
mean,9.233110,0.666667,91.929974,4.336455
std,2.211599,0.487950,9.930071,1.002544
min,6.664744,0.000000,80.009193,2.207028
25%,7.527825,0.000000,80.279478,3.464379
50%,8.085505,1.000000,99.642390,4.621885
75%,10.950501,1.000000,99.775941,4.959160
max,12.948876,1.000000,99.952463,5.926088


In [33]:
a1_index_list = list(a1.index)
a1_x_test_df = X_test_df.iloc[a1_index_list]
len(a1_x_test_df)

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [ ]:
a1_x_test_df.describe()

#### 3 - Anemia of chronic disease

In [34]:
acd_test_df = test_df[test_df.y_actual == 3]
acd_x_test_df = X_test_df[X_test_df.label== 3]
combined_acd_test_df = pd.concat([acd_test_df, acd_x_test_df], axis=1)
combined_acd_test_df.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
1,6.0,1.0,0.0,-1.0,1.0,"['hematocrit', 'gender', 'mcv', 'ferritin', 't...",3.0,7.0,12.609666,104.475099,...,1,0.334722,97.568987,83.161880,-1.000000,7.147270,-1.000000,37.828999,-1.000000,3
15,5.0,15.0,0.0,-1.0,1.0,"['hematocrit', 'gender', 'mcv', 'ferritin', 'I...",3.0,7.0,6.516187,-1.000000,...,0,1.940126,144.949086,110.767294,32.802368,19.240581,82.378455,19.548560,16.372039,3
17,5.0,17.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ferritin', 'A...",3.0,3.0,7.545886,247.939678,...,1,0.488628,43.367298,-1.000000,70.397999,-1.000000,107.084482,22.637657,-1.000000,3
32,5.0,32.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ferritin', 'A...",3.0,3.0,12.079199,460.472174,...,1,0.563605,40.671920,-1.000000,-1.000000,-1.000000,-1.000000,36.237597,-1.000000,3
36,5.0,36.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ferritin', 'A...",3.0,3.0,10.327126,394.552186,...,1,0.542235,89.639054,-1.000000,59.971114,-1.000000,56.192331,30.981377,6.881016,3


In [ ]:
combined_acd_test_df.y_pred.value_counts()

In [ ]:
df_3 = combined_acd_test_df[combined_acd_test_df.y_pred==7]
df_3.head()

In [ ]:
acd_hem_pred_df = combined_acd_test_df[(combined_acd_test_df.y_pred==2)]
acd_hem_pred_df.mcv.mean()

In [ ]:
#rows with tibc in their trajectory
tibc_acd_df = pd.DataFrame()
count=0
for i, row in acd_test_df.iterrows():
    try:
        traj = ast.literal_eval(row.trajectory)
    except:
        traj = row.trajectory
    if 'tibc' in traj:
        tibc_acd_df = tibc_acd_df.append(row)
tibc_acd_df

In [ ]:
X_test_df.loc[6523]

#### 2- Aplastic Anemia

In [ ]:
aplastic_test_df = test_df[test_df.y_pred == 2]
aplastic_x_test_df = X_test_df[X_test_df.label== 2]

In [ ]:
#rows with tibc in their trajectory
neutrophils_aplastic_df = pd.DataFrame()
count=0
for i, row in aplastic_test_df.iterrows():
    try:
        traj = ast.literal_eval(row.trajectory)
    except:
        traj = row.trajectory
    if 'segmented_neutrophils' in traj:
        neutrophils_aplastic_df = neutrophils_aplastic_df.append(row)
neutrophils_aplastic_df

In [ ]:
a2 = aplastic_test_df[aplastic_test_df.y_actual != aplastic_test_df.y_pred]
a2.head()

In [ ]:
a2_index_list = list(a2.index)
a2_x_test_df = X_test_df.iloc[a2_index_list]
a2_x_test_df

In [ ]:
X_test_df.describe()

#### 3 - Iron deficiency anemia 

There are a total of 121 IDA test samples. Of these, 3 are diagnosed as hemolytic anemia while 18 are diagnosed as having aplastic anemia 

In [ ]:
#ida_test_df = test_df[test_df.y_pred == 3]
ida_test_df = test_df[test_df.y_actual == 3]
ida_x_test_df = X_test_df[X_test_df.label== 3]
combined_ida_test_df = pd.concat([ida_x_test_df, ida_test_df], axis=1)
combined_ida_test_df.head()

In [ ]:
combined_ida_test_df.mcv.mean()

In [ ]:
#Looking at the 3 that are diagnoses as hemolytic
ida_hem_pred_df = combined_ida_test_df[(combined_ida_test_df.y_pred==1)|(combined_ida_test_df.y_pred==2)]
ida_hem_pred_df.mcv.mean()

In [ ]:
len(ida_hem_pred_df)

In [ ]:
a3 = ida_test_df[ida_test_df.y_actual != ida_test_df.y_pred]
a3.head()

In [ ]:
#rows with tibc in their trajectory
tibc_ida_df = pd.DataFrame()
count=0
for i, row in ida_test_df.iterrows():
    try:
        traj = ast.literal_eval(row.trajectory)
    except:
        traj = row.trajectory
    if 'tibc' in traj:
        tibc_ida_df = tibc_ida_df.append(row)
tibc_ida_df

In [ ]:
b3_index_list = list(tibc_ida_df.index)
b3_x_test_df = X_test_df.iloc[b3_index_list]
b3_x_test_df